In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as FNN
from torch.autograd import Variable
import csv
from random import shuffle

class Balancer(nn.Module):
    def __init__(self, input_size, hl1_size, hl2_size, output_size):
        super(Balancer, self).__init__()        
        self.Level1 = nn.Linear(input_size, hl1_size)
        self.Level2 = nn.Linear(hl1_size, hl2_size)
        self.Level3 = nn.Linear(hl2_size, output_size)
        
    def forward(self, x):
        var = self.Level1(x)
        var = self.Level2(var)
        var = self.Level3(var)
        return var

class Data():
    def GetData(self, x):
        DataArray = []
        data = open(x)
        data = csv.reader(data)
        return data

dirt = Data()
data = dirt.GetData("balance-scale.data.txt")
AnswerArray = []
TestArray = torch.zeros([625, 4])
TempArray = torch.Tensor([0, 0, 0, 0])
counter = -1
for i in data:
    counter += 1
    if i[0] == "L":                #L=0, B=1, R=2
        AnswerArray.append(0)
    elif i[0] == "B":
        AnswerArray.append(1)
    elif i[0] == "R":
        AnswerArray.append(2)
    for x in range(5):
        if x != 0:
            i[x] = int(i[x])
            TempArray[x-1] = i[x]
            if x==4:
                TestArray[counter][0] = TempArray[0]
                TestArray[counter][1] = TempArray[1]
                TestArray[counter][2] = TempArray[2]
                TestArray[counter][3] = TempArray[3]
                break
            
    
#Starting up the network. Woo!

BalanceNet = Balancer(4, 30, 15, 3)
print(BalanceNet)

LossF1 = nn.CrossEntropyLoss()

OptimF1 = torch.optim.Rprop(BalanceNet.parameters(), lr=0.01, etas=(0.5, 1.2), step_sizes=(1e-06, 50))

BigLoops = 10
CurrentIteration = 0
TestData = torch.zeros(625, 4)
AnswerData = torch.zeros(625, 1)
Indexes = []

for h in range(len(AnswerArray)):
    Indexes.append(h)
    

    
for potato in range(BigLoops):
    NumTried = 0
    NumCorrect = 0
    CurrentIteration = potato
    shuffle(Indexes)
    AnswerData = torch.zeros(625, 1, dtype=torch.long)
    TestData = torch.zeros(625, 4, dtype=torch.float)
    BalanceNet.train()
    OptimF1.zero_grad()
    for l in Indexes:
        AnswerData[l] = AnswerArray[Indexes[l]]
        TestData[l][0] = TestArray[Indexes[l]][0]
        TestData[l][1] = TestArray[Indexes[l]][1]
        TestData[l][2] = TestArray[Indexes[l]][2]
        TestData[l][3] = TestArray[Indexes[l]][3]
    
    guesses = BalanceNet(TestData)  # Do the forward pass
    loss = LossF1(guesses, torch.max(AnswerData, 1)[1]) # Calculate the loss
    loss.backward()       # Calculate the gradients with help of back propagation
    OptimF1.step()      # Ask the optimizer to adjust the parameters based on the gradients
        
    # Record the correct predictions for training data
    NumTried += AnswerData.size(0) 
    _, predicted = torch.max(guesses.data, 1)
    NumCorrect += (predicted == AnswerData.data).sum()

    print ('Epoch %d/%d, Loss: %.4f' 
                %(potato+1, BigLoops, loss.item()))
    BalanceNet.eval()
    
    
    

ExpertData = dirt.GetData('balance-scale-expert.data.txt')
ExAnswerArray = torch.zeros([30, 1])
ExTestArray = torch.zeros([30, 4])
ExTempArray = torch.Tensor([0, 0, 0, 0])
Excounter = -1
for P in ExpertData:
    Excounter += 1
    if P[0] == "L":                
        ExAnswerArray[Excounter] = 0
    elif P[0] == "B":
        ExAnswerArray[Excounter] = 1
    elif P[0] == "R":
        ExAnswerArray[Excounter] = 2
    for U in range(5):
        if U != 0:
            P[U] = int(P[U])
            ExTempArray[U-1] = P[U]
            if U==4:
                ExTestArray[Excounter][0] = ExTempArray[0]
                ExTestArray[Excounter][1] = ExTempArray[1]
                ExTestArray[Excounter][2] = ExTempArray[2]
                ExTestArray[Excounter][3] = ExTempArray[3]
                break
                
print(ExAnswerArray)
print(ExTestArray)
guesses = BalanceNet(ExTestArray)  
loss = LossF1(guesses, torch.max(ExAnswerArray, 1)[1]) 
loss.backward() 
print(loss.item())

Balancer(
  (Level1): Linear(in_features=4, out_features=30, bias=True)
  (Level2): Linear(in_features=30, out_features=15, bias=True)
  (Level3): Linear(in_features=15, out_features=3, bias=True)
)
Epoch 1/10, Loss: 0.3271
Epoch 2/10, Loss: 0.0935
Epoch 3/10, Loss: 0.0197
Epoch 4/10, Loss: 0.0025
Epoch 5/10, Loss: 0.0002
Epoch 6/10, Loss: 0.0000
Epoch 7/10, Loss: 0.0000
Epoch 8/10, Loss: 0.0000
Epoch 9/10, Loss: 0.0000
Epoch 10/10, Loss: 0.0000
tensor([[0.],
        [2.],
        [2.],
        [0.],
        [2.],
        [2.],
        [2.],
        [1.],
        [1.],
        [2.],
        [0.],
        [2.],
        [2.],
        [1.],
        [0.],
        [2.],
        [1.],
        [2.],
        [2.],
        [2.],
        [2.],
        [0.],
        [0.],
        [0.],
        [2.],
        [0.],
        [1.],
        [2.],
        [2.],
        [0.]])
tensor([[  6.,   4.,   2.,   1.],
        [  0.,   0.,   2.,   4.],
        [  6.,   1.,   4.,   3.],
        [ 10.,   2.,   4., 